In [24]:
import requests
import re
import string
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

### Резюме

Ищем ссылки на резюме с ключевым запросом IT, геолокацией -- Россия (по умолчанию отдается Москва), с поиском ключевого запроса (фильтры) 'по всем словам', 'в тексте резюме', пока не начали постоянно отдаваться дубликаты, ограничения на номер страниц в резюме superjob нет

In [53]:
main_url = "https://www.superjob.ru"
search_url = main_url + "/resume/search_resume.html"

resume_urls = np.array([])
all_dublicates_count = 0
i = 1
while all_dublicates_count < 20:
    url_params = {
        'c[0]': '1', # это геолокация == Россия
        'keywords[0][keys]': 'IT',
        'keywords[0][skwc]': 'and',
        'keywords[0][srws]': '7',
        'page': i
    }
    resume_html = requests.get(search_url, params=url_params).text
    soup = BeautifulSoup(resume_html, 'html.parser')
    resume_div = soup.find('div', class_='_1Ttd8 sESpW')
    resume_div = resume_div.find_all('div', class_='_3VcZr')[1]
    resume_divs = resume_div.find_all('div', class_='_1ruK8')
    resume_links = resume_div.find_all('a', class_='icMQ_ f-test-link- _3dPok')
    urls = list(map(lambda link: main_url + link.attrs['href'], resume_links))
    urls = np.array(urls)
    diff = np.unique(np.append(resume_urls, urls)).shape[0] - resume_urls.shape[0] 
    if diff == 0:
        all_dublicates_count += 1
    resume_urls = np.unique(np.append(resume_urls, urls))
    if (i % 10) == 0:
        print("page:", i, "collected:", resume_urls.shape[0])
    i += 1
print("page_count:", i)

page: 10 collected: 300
page: 20 collected: 600
page: 30 collected: 900
page: 40 collected: 1200
page: 50 collected: 1500
page: 60 collected: 1800
page: 70 collected: 2100
page: 80 collected: 2400
page: 90 collected: 2700
page: 100 collected: 3000
page: 110 collected: 3300
page: 120 collected: 3600
page: 130 collected: 3900
page: 140 collected: 4200
page: 150 collected: 4500
page: 160 collected: 4800
page: 170 collected: 5000
page: 180 collected: 5000
page_count: 188


In [52]:
print(len(resume_urls))
resume_urls[:10]

5000


array(['https://www.superjob.ru/resume/3d-modeller-43623349.html',
       'https://www.superjob.ru/resume/administrator-26936498.html',
       'https://www.superjob.ru/resume/administrator-2832565.html',
       'https://www.superjob.ru/resume/administrator-35752711.html',
       'https://www.superjob.ru/resume/administrator-37440177.html',
       'https://www.superjob.ru/resume/administrator-42151588.html',
       'https://www.superjob.ru/resume/administrator-43041335.html',
       'https://www.superjob.ru/resume/administrator-43894667.html',
       'https://www.superjob.ru/resume/administrator-baz-dannyh-9452765.html',
       'https://www.superjob.ru/resume/administrator-it-proekta-44001872.html'],
      dtype='<U117')

In [ ]:
# df = pd.DataFrame(result)
# print(df.shape)
# df.head()

In [ ]:
# with open('data/resume.csv', mode='w', encoding='utf-8') as f_csv:
#     df.to_csv(f_csv)